In [1]:
import nltk 
#nltk.download()
nltk.corpus.brown.words()

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

In [2]:
from nltk.corpus import brown
from nltk.corpus import stopwords
word_list = brown.words()
lower_case_word_list = [word.lower() for word in word_list]

stops = set(stopwords.words("english"))
#print(stops)
filtered_words = [word for word in lower_case_word_list if word not in stops]


In [3]:
import string

print (len(word_list))
print (len(filtered_words))

punctuations = list(string.punctuation)
punctuations.append('``')
punctuations.append('--')
punctuations.append("''")
print (punctuations)

punctuation_filtered_words = [word for word in filtered_words if word not in punctuations]
print (len(punctuation_filtered_words))

1161192
688743
['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '``', '--', "''"]
540871


In [4]:
fdist = nltk.FreqDist(punctuation_filtered_words)
vocab_count = 5000
context_words_count = 1000

# map for each vocab word with corresponding count
vocab_count_map = {}
# set of vocab words for easy lookup
vocab_set = set()

context_count_map = {}
context_set = set()

vocab_with_count = fdist.most_common(vocab_count)
context_words_with_count = fdist.most_common(context_words_count)

for elem in vocab_with_count:
    vocab_set.add(elem[0])
    vocab_count_map[elem[0]] = elem[1]
    
for elem in context_words_with_count:
    context_set.add(elem[0])
    context_count_map[elem[0]] = elem[1]
    
print (len(context_set))

1000


In [5]:
#key of the map is # separated w and c
vocab_context_collocation_map = {}

def populate_vocab_context_map(index):
    if punctuation_filtered_words[index] in context_set:
        key = word + "#" + punctuation_filtered_words[index]
        if key in vocab_context_collocation_map:
            vocab_context_collocation_map[key] += 1
        else:
            vocab_context_collocation_map[key] = 1

# iterate through the pinctuation filtered text stream and populate the vocab_context_collocation_map
for index,word in enumerate(punctuation_filtered_words):
    if word in vocab_set:
        if(index-2 >= 0):
            populate_vocab_context_map(index-2)
        if(index-1 >= 0):
            populate_vocab_context_map(index-1)
        if(index+1 < len(punctuation_filtered_words)):
            populate_vocab_context_map(index+1) 
        if(index+2 < len(punctuation_filtered_words)):
            populate_vocab_context_map(index+2)

print (len(vocab_context_collocation_map))


468130


In [6]:
import math

w, h = 5000, 1000
words_vec_rep = [] 
# calculating point wise mutual information for 5000 vocab words and 1000 context words
for i,vocab in enumerate(vocab_with_count):
    word_vector = []
    for j,context in enumerate(context_words_with_count):
        key = vocab[0]+"#"+context[0]
        colocation_count = 0
        if key in vocab_context_collocation_map:
            colocation_count = vocab_context_collocation_map[key]
        value = (colocation_count * len(punctuation_filtered_words) * 1.0)/(vocab[1]*context[1])
        log_val = 0
        if value>0:
            log_val = math.log(value,2)
        if log_val < 0:
            log_val = 0
        word_vector.append(log_val)
    words_vec_rep.append(word_vector)
    
print (len(words_vec_rep[0]))
#print(words_vec_rep)
        

1000


In [7]:
def dist(vec1, vec2):
    #print(vec1)
    #print(vec2)
    dimension = len(vec1)
    if len(vec1) != len(vec2):
        print(len(vec1))
        print(len(vec2))
    assert len(vec1) == len(vec2)
    sum = 0
    for i in range(dimension):
        sum = sum + float(math.pow((vec1[i]-vec2[i]), 2))
    return math.sqrt(sum)

def determineCluster(vec_word, centers):
    minDist = dist(vec_word, centers[0])
    cluster = 0
    for i in range(1,len(centers)):
        distan = dist(vec_word, centers[i])
        if(distan < minDist):
            cluster = i
            minDist = distan
    return cluster

In [9]:
# k-means clustering algorithm
numClusters = 100
numDim = 100
clusterCenters = words_vec_rep[:numClusters]
#print(len(clusterCenters[0]))
mapOfWordsClassifiedIntoClusters = {}
mapWords = {}
wordsClusters = []
assert len(words_vec_rep) == len(vocab_with_count)
mapOfWordsClassifiedIntoClusters[0] = []
mapWords[0] = []
for i in range(len(words_vec_rep)):
    wordsClusters.append(0)
    mapOfWordsClassifiedIntoClusters[0].append(words_vec_rep[i])
    mapWords[0].append(vocab_with_count[i][0])
for i in range(1,numClusters):
    wordsClusters[i] = i
    mapOfWordsClassifiedIntoClusters[i] = []
    mapOfWordsClassifiedIntoClusters[i].append(words_vec_rep[i])
    mapOfWordsClassifiedIntoClusters[0].remove(words_vec_rep[i])
    mapWords[i] = []
    mapWords[i].append(vocab_with_count[i][0])
    mapWords[0].remove(vocab_with_count[i][0])
converged = False
numiter = 0
while not converged or numiter == 0:
    numiter = numiter+1
    print(numiter)
    converged = True
    for i, wordVec in enumerate(words_vec_rep):
        # its vector representation is words_vec_rep[i] - calculate its distance with 100 centers and assign to closest
        clus = determineCluster(wordVec, clusterCenters)
        assert clus < numClusters
        mapOfWordsClassifiedIntoClusters[wordsClusters[i]].remove(wordVec)
        mapOfWordsClassifiedIntoClusters[clus].append(wordVec)
        mapWords[wordsClusters[i]].remove(vocab_with_count[i][0])
        mapWords[clus].append(vocab_with_count[i][0])
        wordsClusters[i] = clus
    
    #for i in range(numClusters):
     #   print('cluster: ', i, 'words: ', mapWords[i])
        
    for i in range(numClusters):
        newCenterVector = [0]*numDim
        for wordVec in mapOfWordsClassifiedIntoClusters[i]:
            # calculate 100 means and update cluster center i
            for dim in range(numDim):
                newCenterVector[dim] += wordVec[dim]
        for dim in range(numDim):
            newCenterVector[dim] /= len(mapOfWordsClassifiedIntoClusters[i])
        if clusterCenters[i] != newCenterVector:
            clusterCenters[i] = newCenterVector
            converged = False

print (clusterCenters)
for i in range(numClusters):
     print('cluster: ', i, 'words: ', mapWords[i])
#print (mapWords[50])
        
        


1


KeyboardInterrupt: 

In [16]:
from sklearn.cluster import KMeans
import numpy as np
from sklearn.decomposition import PCA
kmeans = KMeans(n_clusters=numClusters, random_state=0)


# words have 1000 dimensional representation - reduce this to 100 dimension by SVD
pca = PCA(n_components=100)
pca.fit(words_vec_rep)
lessDimWordRep = pca.transform(words_vec_rep)
print(pca.explained_variance_ratio_) 
print(lessDimWordRep[0])

kmeans.fit(lessDimWordRep)
labels = kmeans.predict(lessDimWordRep)
#kmeans.predict(words_vec_rep)
centers = kmeans.cluster_centers_


[ 0.03684536  0.01807526  0.00820909  0.00652986  0.00516692  0.00465679
  0.00372454  0.00334725  0.00294418  0.00287741  0.00279013  0.00266506
  0.00262567  0.00251635  0.00244988  0.00243611  0.00242653  0.00235834
  0.00231995  0.00226368  0.0022096   0.00220518  0.00217178  0.00214658
  0.00213697  0.00211818  0.00209578  0.00207367  0.00206357  0.00205286
  0.002032    0.0020183   0.00199438  0.00199101  0.00198218  0.00197535
  0.00196129  0.00194389  0.00193812  0.00193019  0.00192488  0.00191865
  0.0019152   0.00191181  0.00190355  0.00189399  0.00188652  0.00188239
  0.0018737   0.00186732  0.00185985  0.00185251  0.00184851  0.00184401
  0.00183991  0.00183859  0.00182565  0.00181748  0.00181238  0.00180696
  0.00180465  0.00179796  0.00179538  0.00179079  0.00178301  0.00178033
  0.00177644  0.0017712   0.00176563  0.00176165  0.00175239  0.00174728
  0.0017435   0.00173987  0.00173363  0.00173071  0.00172972  0.0017231
  0.00171729  0.00171373  0.00170846  0.00170467  0.

In [67]:
mapClusters = {}
#print(labels)
for i in range(len(words_vec_rep)):
    clus = labels[i]
    if clus not in mapClusters.keys():
        mapClusters[clus] = []
    mapClusters[clus].append(vocab_with_count[i][0])
    
for i in range(numClusters):
    if(len(mapClusters[i]) < 20):
        print( i,mapClusters[i])

5 ['bill', 'served', 'health', 'becomes', 'prepared', 'occurred', 'providing', 'introduced']
7 ['money', 'available', 'cost', 'tax', 'required', 'pay', 'amount', 'paid', 'additional', 'income', 'current']
12 ['small', 'white', 'line', 'usually', 'women', 'dark']
22 ['shall', 'purpose', 'due', 'provided', 'limited', 'claim']
24 ['service', 'special', 'military', 'class', 'medical', 'aid']
25 ['state', 'public', 'city', 'national', 'members', 'area', 'local', 'labor', 'planning']
28 ['heard', 'leave', 'says', 'tried', 'happened', 'ready', 'try', 'remember', 'trouble', 'chance', 'ask', 'easy', 'stop', 'hit', "there's", 'suppose', 'someone', 'nobody']
33 ['information', 'needed', 'methods', 'services', 'systems', 'types', 'parts', 'factors', 'behavior', 'purposes', 'sources', 'requirements', 'items', 'resources', 'levels', 'adequate']
34 ['peace', 'recent', 'nations', 'needs', 'countries', 'trade', 'throughout', 'nation', 'soviet', 'freedom', 'efforts', 'progress', 'leaders', 'relations', 

In [66]:
from sklearn.metrics.pairwise import cosine_similarity
cos = cosine_similarity(words_vec_rep)
#print(cos[0])
#print(cos[5000-1][5000-1])
'''
def dist_nn(vec1, vec2):
    assert len(vec1) == len(vec2)
    norm1 = 0
    norm2 = 0
    for i in range(len(vec1)):
        norm1 = norm1 + vec1[i]*vec1[i]
        norm2 = norm2 + vec2[i]*vec2[i]
    normProduct = math.sqrt(norm1)*math.sqrt(norm2)
    dot = 0
    for i in range(len(vec1)):
        dot = dot + vec1[i]*vec2[i]
    return (1-(dot/normProduct))

def find_NN(word_vec):
    minDist = dist_nn(word_vec, words_vec_rep[-1])
    print('see', minDist)
    assert not minDist == 0
    index = len(words_vec_rep)-1
    for i, wo in enumerate(words_vec_rep):
        #if word_vec == wo:
         #   continue
        #print(word_vec)
        #print(wo)
        equal = True
        for j in range(len(word_vec)):
            if not (word_vec[j] == wo[j]):
                equal = False
                break
        if equal:
            continue
        distan = dist_nn(word_vec, wo)
        if distan > minDist:
            #print('oops',distan)
            minDist = distan
            index = i
    return index
'''
def find_NN_New(idx):
    # find max in that row
    maxSim = 0
    index = -1
    for i in range(5000):
        if i == idx:
            continue
        if cos[i][idx] > maxSim:
            index = i
            maxSim = cos[i][idx]
    return index
# nearest neighbors
#selected_vocab = vocab_with_count[:25]

selected_vocab = ['communism', 'autumn', 'cigarette', 'pulmonary', 'mankind',\
                  'africa', 'chicago', 'revolution', 'september', 'chemical', \
                  'detergent', 'dictionary', 'storm', 'worship', 'god', \
                  'population', 'fifteen', 'leaders', 'hands', 'january', \
                  'information', 'students', 'years', 'england', 'philosophy' ]

indices = [0]*25
for i, wr in enumerate(vocab_with_count):
    if 'concerned' == wr[0]:
        print('concerned',i)
    if 'communism' == wr[0]:
        print('comm',i)
        
    if wr[0] in selected_vocab:
        #print(wr[0])
        idx = selected_vocab.index(wr[0])
        indices[idx] = i
        #print('see', i, idx)

nn = []
for i,j in enumerate(vocab_context_collocation_map.keys()):
    if 'communism' in j:
        print(j, vocab_context_collocation_map[j])

#print(cos[634][1422])
for i in range(5000):
    print(cos[i][1422])
#print(vocab_with_count[5000])
#vocab_context_collocation_map
for i, word_nn in enumerate(selected_vocab):
    nearest_word = vocab_with_count[find_NN_New(indices[i])][0]
    nn.append(nearest_word)
    key = word_nn + '#' + nearest_word
    a = 0
    if key in vocab_context_collocation_map.keys():
        a = vocab_context_collocation_map[key]
    key = nearest_word + '#' + word_nn
    if key in vocab_context_collocation_map.keys():
        b = vocab_context_collocation_map[key]
    print( i+1,',', word_nn,',', nearest_word,',', a )

concerned 634
comm 1422
communism#never 1
communism#either 1
communism#would 2
communism#lay 1
communism#total 1
communism#three 1
communism#full 1
communism#turn 1
communism#since 1
communism#situation 1
communism#mass 1
communism#city 1
communism#america 1
communism#high 1
communism#concerned 5
communism#air 1
communism#although 2
communism#war 4
communism#designed 1
communism#clear 1
communism#speak 2
communism#nothing 2
communism#hundred 1
communism#local 1
communism#last 1
communism#world 2
communism#nuclear 1
communism#going 1
communism#half 1
communism#modern 3
communism#book 1
communism#answer 1
communism#past 1
communism#public 1
communism#find 2
communism#also 2
communism#years 1
communism#least 1
communism#indicated 1
communism#congress 1
communism#toward 1
communism#new 2
communism#building 1
communism#things 1
communism#old 1
communism#idea 1
communism#want 1
communism#make 1
communism#history 1
communism#problems 1
communism#school 2
communism#led 1
communism#red 1
commun

In [57]:
print(selected_vocab)

[('one', 3292), ('would', 2714), ('said', 1961), ('new', 1635), ('could', 1601), ('time', 1598), ('two', 1412), ('may', 1402), ('first', 1361), ('like', 1292), ('man', 1207), ('even', 1170), ('made', 1125), ('also', 1069), ('many', 1030), ('must', 1013), ('af', 996), ('back', 966), ('years', 950), ('much', 937), ('way', 908), ('well', 897), ('people', 847), ('mr.', 844), ('little', 831)]
